# DATA CLEANING

In [2]:
import pandas as pd
import numpy as np

data_set = pd.read_csv('airbnb_lisbon_1480_2017-07-27.csv')
data_set.head()

data_set2 = data_set.apply(lambda x: x.astype(str).str.lower()) #this converts dataframe to lowercase
col_to_drop = ["country", "borough", "room_id", "survey_id", "host_id", "bathrooms", "minstay", "last_modified","latitude","longitude","location"]
my_data = data_set2.drop(col_to_drop, axis=1)
my_data[['reviews','overall_satisfaction','price']] = my_data[['reviews','overall_satisfaction','price']].astype('float')
#my_data.dtypes


#dropped_col_data = dropped_col_data[dropped_col_data.reviews !=0]
new_data = my_data[my_data["reviews"] != 0]

new_data.tail(20)
new_data.to_csv('cleaned_data.csv', index=False)

# IMPORT CLEAN DATA

In [1]:
import pandas as pd
import numpy as np

clean_data = pd.read_csv('cleaned_data.csv')
clean_data.head()

,room_type,city,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,name
0,shared room,lisbon,santo antónio,19.0,4.5,4,1.0,30.0,low cost hostel marques gardens 4 bed dorm
1,shared room,lisbon,avenidas novas,4.0,3.5,6,1.0,39.0,room in lisbon center
2,shared room,lisbon,santa maria maior,38.0,4.5,8,1.0,39.0,bed in a 8-bed dorm @ this is lisbon hostel
3,shared room,lisbon,avenidas novas,9.0,4.0,6,1.0,37.0,shared mixed bunkbed room 1
4,shared room,lisbon,arroios,1.0,0.0,2,1.0,96.0,nice room anjos


# DATA FOR RATINGS

In [2]:
drop_list = ["city", "accommodates", "name", "room_type", "bedrooms", "price"]
ratings = clean_data.drop(drop_list, axis = 1)
ratings['ratings_weight'] = (ratings['reviews']*(0.4)) + (ratings['overall_satisfaction']*(0.6))
ratings.head()

ratings_agg = round((ratings.groupby(['neighborhood']).agg(['mean'])),2)
ratings_agg["reviews"] = ratings_agg['reviews'].astype('int')
ratings_agg = ratings_agg.reset_index(level = 0)
ratings_agg.columns = ["Neighborhood", "Review Quantity", "Overall Satisfaction", "Weighted Ratings"]
ratings_agg["Rank"] = ratings_agg['Weighted Ratings'].rank(ascending = False).astype('int')
ratings_agg = ratings_agg[['Rank',"Neighborhood","Weighted Ratings", "Review Quantity","Overall Satisfaction"]]
ratings_agg.head(50)
ratings_agg.sort_values(["Weighted Ratings"],ascending = True)

#ratings_mean.sort_values(['overall_satisfaction'], ascending=False)

,Rank,Neighborhood,Weighted Ratings,Review Quantity,Overall Satisfaction
11,24,carnide,4.34,7,2.57
19,23,santa clara,4.72,6,3.42
17,22,parque das nações,6.78,11,3.46
13,21,lumiar,7.42,13,3.17
10,20,campolide,7.50,13,3.55
8,19,benfica,7.99,14,3.96
22,18,são domingos de benfica,8.97,17,3.40
3,17,areeiro,9.29,18,3.36
14,16,marvila,9.57,18,3.79
5,15,avenidas novas,9.64,19,3.30


# DATA FOR PRICE

In [3]:
drop_list = ["city", "accommodates", "name", "reviews", "overall_satisfaction","room_type"]
price = clean_data.drop(drop_list, axis = 1)
price['bedrooms'] = price['bedrooms'].map(lambda x: 1 if x==0 else x)
price['price/br'] = price['price']/price["bedrooms"]
drop_price = ["price","bedrooms"]
price = price.drop(drop_price, axis = 1)
price_agg = round(price.groupby(['neighborhood']).agg(['mean']),2)
price_agg = price_agg.reset_index(level = 0)
price_agg.columns = ["Neighborhood","Price / BR"]
price_agg["Rank"] = price_agg['Price / BR'].rank(ascending = True).astype('int')
price_agg = price_agg[["Rank","Neighborhood","Price / BR"]]
price_agg.sort_values('Price / BR',ascending=False)



#price.groupby(['neighborhood']).agg(['mean'])
#price.head(100)

,Rank,Neighborhood,Price / BR
17,24,parque das nações,64.26
20,23,santa maria maior,58.98
15,22,misericórdia,57.29
7,21,belém,56.90
23,20,são vicente,52.12
12,19,estrela,52.11
21,18,santo antónio,52.05
9,17,campo de ourique,50.14
1,16,alcântara,49.00
11,15,carnide,47.00


# MERGE RATINGS & PRICE

In [4]:
ratings_price_agg = ratings_agg.merge(price_agg, on = 'Neighborhood')
ratings_price_agg.columns = ["Ratings Rank","Neighborhood","Weighted Ratings","Review Quantity","Overall Satisfaction","Price Rank","Price / BR"]
ratings_price_agg.sort_values("Ratings Rank")

,Ratings Rank,Neighborhood,Weighted Ratings,Review Quantity,Overall Satisfaction,Price Rank,Price / BR
20,1,santa maria maior,22.38,49,4.14,23,58.98
15,2,misericórdia,20.53,45,4.07,22,57.29
16,3,olivais,18.86,40,4.13,10,42.98
23,4,são vicente,17.00,36,4.10,20,52.12
12,5,estrela,15.77,33,4.00,19,52.11
21,6,santo antónio,15.55,32,3.95,18,52.05
4,7,arroios,13.24,27,3.82,11,43.85
2,8,alvalade,12.04,24,3.90,7,40.37
18,9,penha de frança,11.56,23,3.75,3,37.18
6,10,beato,11.52,23,3.55,6,39.59


In [5]:
ratings_price_agg["Weighted Ratings"].corr(ratings_price_agg["Price / BR"])

0.40228992753471937

# --------------------------------------------------------------------------------------------------

# BAG OF WORDS

In [6]:
clean_data.head()

,room_type,city,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,price,name
0,shared room,lisbon,santo antónio,19.0,4.5,4,1.0,30.0,low cost hostel marques gardens 4 bed dorm
1,shared room,lisbon,avenidas novas,4.0,3.5,6,1.0,39.0,room in lisbon center
2,shared room,lisbon,santa maria maior,38.0,4.5,8,1.0,39.0,bed in a 8-bed dorm @ this is lisbon hostel
3,shared room,lisbon,avenidas novas,9.0,4.0,6,1.0,37.0,shared mixed bunkbed room 1
4,shared room,lisbon,arroios,1.0,0.0,2,1.0,96.0,nice room anjos


In [43]:
drop_list = ["city", "accommodates", "room_type", "bedrooms", "price"]
bow_ratings = clean_data.drop(drop_list, axis = 1)
bow_ratings['ratings_weight'] = (ratings['reviews']*(0.4)) + (ratings['overall_satisfaction']*(0.6))
bow_ratings.sort_values('ratings_weight', ascending = False).head()

# Limit to weighting over 100
ratings_100 = bow_ratings[bow_ratings["neighborhood"] == 'santa maria maior']
ratings_100.head()

,neighborhood,reviews,overall_satisfaction,name,ratings_weight
2,santa maria maior,38.0,4.5,bed in a 8-bed dorm @ this is lisbon hostel,17.9
17,santa maria maior,2.0,0.0,lisbon cozy flat - bunk beds 5,0.8
29,santa maria maior,7.0,4.5,lisbon destination hostel - dorm beds,5.5
32,santa maria maior,1.0,0.0,lisbon cozy flat - bunk beds 2,0.4
37,santa maria maior,3.0,3.0,twin room (dormitory style),3.0


In [44]:
corpus = ratings_100["name"].values
corpus


array(['bed in a 8-bed dorm @ this is lisbon hostel',
       'lisbon cozy flat - bunk beds 5',
       'lisbon destination hostel - dorm beds', ...,
       'a safe and lovely spot downtown',
       'quarto simples para uma pessoa',
       'luminous suite @city centre - baixa chiado'], dtype=object)

In [45]:
import sklearn.feature_extraction.text
from sklearn.feature_extraction.text import CountVectorizer

stop_words = ['all', 'six', 'less', 'being', 'indeed', 'over', 'move', 'anyway', 'fifty', 'four', 'not', 'own', 'through', 'yourselves', 'go', 'where', 'mill', 'only', 'find', 'before', 'one', 'whose', 'system', 'how', 'somewhere', 'with', 'thick', 'show', 'had', 'enough', 'should', 'to', 'must', 'whom', 'seeming', 'under', 'ours', 'has', 'might', 'thereafter', 'latterly', 'do', 'them', 'his', 'around', 'than', 'get', 'very', 'de', 'none', 'cannot', 'every', 'whether', 'they', 'front', 'during', 'thus', 'now', 'him', 'nor', 'name', 'several', 'hereafter', 'always', 'who', 'cry', 'whither', 'this', 'someone', 'either', 'each', 'become', 'thereupon', 'sometime', 'side', 'two', 'therein', 'twelve', 'because', 'often', 'ten', 'our', 'eg', 'some', 'back', 'up', 'namely', 'towards', 'are', 'further', 'beyond', 'ourselves', 'yet', 'out', 'even', 'will', 'what', 'still', 'for', 'bottom', 'mine', 'since', 'please', 'forty', 'per', 'its', 'everything', 'behind', 'un', 'above', 'between', 'it', 'neither', 'seemed', 'ever', 'across', 'she', 'somehow', 'be', 'we', 'full', 'never', 'sixty', 'however', 'here', 'otherwise', 'were', 'whereupon', 'nowhere', 'although', 'found', 'alone', 're', 'along', 'fifteen', 'by', 'both', 'about', 'last', 'would', 'anything', 'via', 'many', 'could', 'thence', 'put', 'against', 'keep', 'etc', 'amount', 'became', 'ltd', 'hence', 'onto', 'or', 'con', 'among', 'already', 'co', 'afterwards', 'formerly', 'within', 'seems', 'into', 'others', 'while', 'whatever', 'except', 'down', 'hers', 'everyone', 'done', 'least', 'another', 'whoever', 'moreover', 'couldnt', 'throughout', 'anyhow', 'yourself', 'three', 'from', 'her', 'few', 'together', 'top', 'there', 'due', 'been', 'next', 'anyone', 'eleven', 'much', 'call', 'therefore', 'interest', 'then', 'thru', 'themselves', 'hundred', 'was', 'sincere', 'empty', 'more', 'himself', 'elsewhere', 'mostly', 'on', 'fire', 'am', 'becoming', 'hereby', 'amongst', 'else', 'part', 'everywhere', 'too', 'herself', 'former', 'those', 'he', 'me', 'myself', 'made', 'twenty', 'these', 'bill', 'cant', 'us', 'until', 'besides', 'nevertheless', 'below', 'anywhere', 'nine', 'can', 'of', 'your', 'toward', 'my', 'something', 'and', 'whereafter', 'whenever', 'give', 'almost', 'wherever', 'is', 'describe', 'beforehand', 'herein', 'an', 'as', 'itself', 'at', 'have', 'in', 'seem', 'whence', 'ie', 'any', 'fill', 'again', 'hasnt', 'inc', 'thereby', 'thin', 'no', 'perhaps', 'latter', 'meanwhile', 'when', 'detail', 'same', 'wherein', 'beside', 'also', 'that', 'other', 'take', 'which', 'becomes', 'you', 'if', 'nobody', 'see', 'though', 'may', 'after', 'upon', 'most', 'hereupon', 'eight', 'but', 'serious', 'nothing', 'such', 'why', 'a', 'off', 'whereby', 'third', 'i', 'whole', 'noone', 'sometimes', 'well', 'amoungst', 'yours', 'their', 'rather', 'without', 'so', 'five', 'the', 'first', 'whereas', 'once']

vectorizer = CountVectorizer(stop_words=stop_words)

x = vectorizer.fit_transform(corpus)

bow = vectorizer.vocabulary_
print(bow)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [37]:
term_freq = x.toarray()
print(term_freq)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [38]:
bow_100 = pd.DataFrame(list(bow.items()), columns = ['Word',"Count"])
bow_100["Rank"] = bow_100['Count'].rank(ascending=False).astype('int')
bow_100 = bow_100[["Rank","Word","Count"]]
bow_100.sort_values("Count", ascending=False)

#bow_100.sort_values("Count", ascending=False)

,Rank,Word,Count
76,1,yard,108
50,2,wonderful,107
65,3,wifi,106
32,4,white,105
66,5,vista,104
42,6,vintage,103
11,7,view,102
72,8,typical,101
69,9,tiles,100
7,10,terrace,99


# BAG OF WORDS - REFERENCE

In [ ]:
corpus = np.array(())
corpus = (np.array(np.concatenate([open(x, "r").readlines() for x in name_agg]))).tolist()
print(corpus)
vectorizer = CountVectorizer()

x = vectorizer.fit_transform(corpus)

bow = vectorizer.vocabulary_
print(bow)

# bag_of_words = vectorizer.get_feature_names()
# print(bag_of_words)

term_freq = x.toarray()
print(term_freq)


#corpus = sklearn.feature_extraction.text.CountVectorizer(docs, stop_words = stop_words)
#print(corpus)

# --------------------------------------------------------------------------------------------------

# DATA FOR ROOMTYPE

In [34]:
drop_list = ["city", "accommodates", "name", "reviews", "overall_satisfaction"]
price = clean_data.drop(drop_list, axis = 1)
price.head()

,room_type,neighborhood,bedrooms,price
0,shared room,santo antónio,1.0,30.0
1,shared room,avenidas novas,1.0,39.0
2,shared room,santa maria maior,1.0,39.0
3,shared room,avenidas novas,1.0,37.0
4,shared room,arroios,1.0,96.0


In [36]:
price['price/br'] = price['price']/price["bedrooms"]
price.head(20)

,room_type,neighborhood,bedrooms,price,price/br
0,shared room,santo antónio,1.0,30.0,30.0
1,shared room,avenidas novas,1.0,39.0,39.0
2,shared room,santa maria maior,1.0,39.0,39.0
3,shared room,avenidas novas,1.0,37.0,37.0
4,shared room,arroios,1.0,96.0,96.0
5,shared room,campo de ourique,1.0,90.0,90.0
6,shared room,lumiar,1.0,96.0,96.0
7,shared room,parque das nações,1.0,84.0,84.0
8,shared room,avenidas novas,1.0,66.0,66.0
9,shared room,areeiro,1.0,58.0,58.0


# SWITCH ROOM TYPE TO VARIABLE

In [39]:
drop_price = ["price"]
price2 = price.drop(drop_price, axis = 1)
price3 = price2.pivot_table(index = ["neighborhood","bedrooms"], columns = ["room_type"])
price3.head(20)

price/br                         
room_type             entire home/apt private room shared room
neighborhood bedrooms                                         
ajuda        0.0                  inf          NaN         NaN
             1.0            51.041667    30.523810   18.000000
             2.0            34.825000          NaN         NaN
             3.0            32.000000          NaN         NaN
             4.0            86.250000          NaN         NaN
             9.0            25.444444          NaN         NaN
alcântara    0.0                  inf          NaN         NaN
             1.0            59.704918    42.970588         NaN
             2.0            44.818182          NaN         NaN
             3.0            28.777778          NaN         NaN
             4.0            33.416667          NaN         NaN
             6.0            40.000000          NaN         NaN
alvalade     0.0                  inf          inf         NaN
             1.0            62.218750    34.643836   29.500000
             2.0            36.775862    22.000000         NaN
             3.0            37.100000          NaN         NaN
             4.0            21.083333          NaN         NaN
             5.0            49.900000          NaN         NaN
areeiro      0.0                  NaN          inf         NaN
             1.0            70.325000    32.312500   20.642857